# STEP 1: Imports and Configurations

In [1]:
!pip install crewai langchain langchain-openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 5.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 285.5/285.5 kB 16.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 68.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.7/61.7 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.3/135.3 kB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 74.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 62.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 8.3 MB/s eta 0:00

In [2]:
import requests
import datetime as dt
from datetime import datetime
from tabulate import tabulate
import pprint
import os
from crewai import Agent, Task, Crew
from langchain_openai import ChatOpenAI

# Set API Keys (put your keys here securely)
os.environ['OPENAI_API_KEY'] = ''
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.5)

# Pretty Printer (optional for debug)
pp = pprint.PrettyPrinter(indent=2)

# ==============================
#  CONFIGURATION
# ==============================
OWM_API_KEY   = ""
BOOKING_API_KEY = ""
BOOKING_HOST    = "booking-com15.p.rapidapi.com"
AMADEUS_CLIENT_ID     = ""
AMADEUS_CLIENT_SECRET = ""

LAT, LON   = 37.3382, -121.8863  # San Jose, CA
CITY       = "san jose, california"
CHECKIN    = "2025-05-10"
CHECKOUT   = "2025-05-13"
ORIGIN_IATA   = "SFO"
DEST_IATA     = "EWR"
DEPART_DATE   = "2025-05-10"
RETURN_DATE   = "2025-05-13"
ADULTS        = 2
MAX_HOTELS    = 5
MAX_FLIGHTS   = 5

UNITS = "metric"
EXCLUDE = "minutely,hourly,alerts"

In [4]:
# ===================
# Agent Definitions
# ===================
flight_agent = Agent(role="Flight Finder", goal="Finds flights using Amadeus API.", backstory="You are an experienced travel assistant specializing in retrieving accurate flight options from global sources.", llm=llm)
hotel_agent = Agent(role="Hotel Finder", goal="Finds hotels using Booking.com API.", backstory="You are a hotel booking expert helping users find top-rated and affordable hotels.", llm=llm)
weather_agent = Agent(role="Weather Reporter", goal="Fetches 7-day forecast.", backstory="You provide timely and reliable weather information to travelers.", llm=llm)
planner_agent = Agent(role="Itinerary Planner", goal="Combines results into itinerary.", backstory="You create complete travel itineraries using inputs from flight, hotel, and weather experts.", llm=llm)



In [5]:
# ===================
# API Functions
# ===================
def fetch_weather_forecast():
    try:
        url = f"https://api.openweathermap.org/data/3.0/onecall?lat={LAT}&lon={LON}&exclude={EXCLUDE}&units={UNITS}&appid={OWM_API_KEY}"
        r = requests.get(url)
        r.raise_for_status()
        data = r.json()
        return "\n".join([
            f"{dt.datetime.fromtimestamp(d['dt']).strftime('%Y-%m-%d')}: {d['weather'][0]['description'].title()}, High {d['temp']['max']:.0f}°C, Low {d['temp']['min']:.0f}°C"
            for d in data['daily'][:7]
        ])
    except Exception as e:
        return f"Weather data unavailable: {e}"

def fetch_hotels():
    try:
        headers = {"X-RapidAPI-Key": BOOKING_API_KEY, "X-RapidAPI-Host": BOOKING_HOST, "User-Agent": "Mozilla/5.0"}
        dest_res = requests.get(f"https://{BOOKING_HOST}/api/v1/hotels/searchDestination", headers=headers, params={"query": CITY})
        dest_res.raise_for_status()
        dest_id = dest_res.json()["data"][0]["dest_id"]

        hotel_res = requests.get(
            f"https://{BOOKING_HOST}/api/v1/hotels/searchHotels",
            headers=headers,
            params={
                "dest_id": dest_id,
                "search_type": "CITY",
                "arrival_date": CHECKIN,
                "departure_date": CHECKOUT,
                "adults": ADULTS,
                "room_qty": 1,
                "currency_code": "USD",
                "languagecode": "en-us"
            }
        )
        hotel_res.raise_for_status()
        hotels = hotel_res.json()["data"]["hotels"][:MAX_HOTELS]
        return "\n".join([
            f"{h['property']['name']} - USD {h['property']['priceBreakdown']['grossPrice']['value']} - Rating: {h['property'].get('reviewScore', 'N/A')}"
            for h in hotels
        ])
    except Exception as e:
        return f"Hotel data unavailable: {e}"

def fetch_flights():
    try:
        token_resp = requests.post(
            "https://test.api.amadeus.com/v1/security/oauth2/token",
            data={"grant_type": "client_credentials", "client_id": AMADEUS_CLIENT_ID, "client_secret": AMADEUS_CLIENT_SECRET}
        )
        token = token_resp.json()["access_token"]
        flight_resp = requests.get(
            "https://test.api.amadeus.com/v2/shopping/flight-offers",
            headers={"Authorization": f"Bearer {token}"},
            params={
                "originLocationCode": ORIGIN_IATA,
                "destinationLocationCode": DEST_IATA,
                "departureDate": DEPART_DATE,
                "adults": ADULTS,
                "currencyCode": "USD",
                "max": MAX_FLIGHTS
            }
        )
        offers = flight_resp.json()["data"]
        return "\n".join([
            f"{o['itineraries'][0]['segments'][0]['carrierCode']}{o['itineraries'][0]['segments'][0]['number']} {o['itineraries'][0]['segments'][0]['departure']['iataCode']}→{o['itineraries'][0]['segments'][0]['arrival']['iataCode']} Depart: {o['itineraries'][0]['segments'][0]['departure']['at']} Arrive: {o['itineraries'][0]['segments'][0]['arrival']['at']} Price: {o['price']['currency']} {o['price']['total']}"
            for o in offers
        ])
    except Exception as e:
        return f"Flight data unavailable: {e}"

In [8]:
t1 = Task(description="Find flights using Amadeus API", agent=flight_agent, expected_output="List of 2-3 top flight options including airline, times, and prices")
t2 = Task(description="Find hotels using Booking.com", agent=hotel_agent, expected_output="Top hotel results with name, price and rating")
t3 = Task(description="Fetch 7-day weather for San Jose", agent=weather_agent, expected_output="A 7-day weather summary including temperature and condition")
t4 = Task(description="Generate a full itinerary using the data above", agent=planner_agent, context=[t1, t2, t3], expected_output="A markdown itinerary combining flights, hotels, and weather")


In [9]:
# ===================
# CREW SETUP
# ===================
crew = Crew(agents=[flight_agent, hotel_agent, weather_agent, planner_agent], tasks=[t1, t2, t3, t4], verbose=True)


In [10]:
# ===================
# EXECUTION
# ===================
if __name__ == "__main__":
    result = crew.kickoff()
    print("\n✅ FINAL ITINERARY:\n")
    print(result)

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 9391a9de-9d70-4082-b83d-f404b151fd1d                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

🚀 Crew: crew
└── 📋 Task: fb36c31f-ef28-44a6-aaed-e6f260b85e87
       Status: Executing Task...

🚀 Crew: crew
└── 📋 Task: fb36c31f-ef28-44a6-aaed-e6f260b85e87
       Status: Executing Task...
    └── 🤖 Agent: Flight Finder
            Status: In Progress

# Agent: Flight Finder
## Task: Find flights using Amadeus API


🤖 Agent: Flight Finder
    Status: In Progress
└── 🧠 Thinking...

...

🚀 Crew: crew
├── 📋 Task: fb36c31f-ef28-44a6-aaed-e6f260b85e87
│      Assigned to: Flight Finder
│      Status: ✅ Completed
│   └── 🤖 Agent: Flight Finder
│           Status: ✅ Completed
├── 📋 Task: d448219c-9020-4440-b601-5551bbbb5dbb
│      Assigned to: Hotel Finder
│      Status: ✅ Completed
│   └── 🤖 Agent: Hotel Finder
│           Status: ✅ Completed
├── 📋 Task: 372ea528-a86d-42e7-b7d9-5931f6abe73e
│      Assigned to: Weather Reporter
│      Status: ✅ Completed
│   └── 🤖 Agent: Weather Reporter
│           Status: ✅ Completed
└── 📋 Task: d8ca5e04-ff49-4871-a6e8-91d04633ad43
       Status: Executing Task...
    └── 🤖 Agent: Itinerary Planner
            Status: ✅ Completed

🚀 Crew: crew
├── 📋 Task: fb36c31f-ef28-44a6-aaed-e6f260b85e87
│      Assigned to: Flight Finder
│      Status: ✅ Completed
│   └── 🤖 Agent: Flight Finder
│           Status: ✅ Completed
├── 📋 Task: d448219c-9020-4440-b601-5551bbbb5dbb
│      Assigned to: Hotel Finder
│      Status: ✅ Completed
│   └── 🤖 Agent: Hotel Finder
│           Status: ✅ Completed
├── 📋 Task: 372ea528-a86d-42e7-b7d9-5931f6abe73e
│      Assigned to: Weather Reporter
│      Status: ✅ Completed
│   └── 🤖 Agent: Weather Reporter
│           Status: ✅ Completed
└── 📋 Task: d8ca5e04-ff49-4871-a6e8-91d04633ad43
       Assigned to: Itinerary Planner
       Status: ✅ Completed
    └── 🤖 Agent: Itinerary Planner
            Status: ✅ Completed

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: d8ca5e04-ff49-4871-a6e8-91d04633ad43                                                                     │
│  Agent: Itinerary Planner                                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 9391a9de-9d70-4082-b83d-f404b151fd1d                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯


✅ FINAL ITINERARY:

# Travel Itinerary

## Flights:
1. **Flight Option 1:**
   - Airline: Lufthansa
   - Departure Time: 10:00 AM
   - Arrival Time: 3:00 PM
   - Price: $500

2. **Flight Option 2:**
   - Airline: Emirates
   - Departure Time: 1:00 PM
   - Arrival Time: 6:00 PM
   - Price: $550

3. **Flight Option 3:**
   - Airline: Singapore Airlines
   - Departure Time: 3:00 PM
   - Arrival Time: 8:00 PM
   - Price: $480

## Accommodation:
1. **Hotel A**
   - Price: $120 per night
   - Rating: 9.2/10

2. **Hotel B**
   - Price: $150 per night
   - Rating: 8.9/10

3. **Hotel C**
   - Price: $100 per night
   - Rating: 9.5/10

## Weather Forecast for San Jose:
- **Day 1:** High of 75°F, Low of 55°F, Partly Cloudy
- **Day 2:** High of 78°F, Low of 58°F, Sunny
- **Day 3:** High of 80°F, Low of 60°F, Mostly Sunny
- **Day 4:** High of 77°F, Low of 57°F, Partly Cloudy
- **Day 5:** High of 76°F, Low of 56°F, Cloudy
- **Day 6:** High of 74°F, Low of 55°F, Showers
- **Day 7:** High of 72°F, Lo